In [2]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# librosa is a Python library for analyzing audio and music.
# It can be used to extract the data from the audio files we will see it later
import librosa 
import librosa.display

# to play the audio files
from IPython.display import Audio
# plt.style.use('seaborn-white')

fem_path = './Female_features.csv'
mal_path = './Male_features.csv'

Females_Features = pd.read_csv(fem_path)
Males_Features = pd.read_csv(mal_path)


from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


female_X = Females_Features.iloc[: ,:-1].values
female_Y = Females_Features['labels'].values

male_X = Males_Features.iloc[: ,:-1].values
male_Y = Males_Features['labels'].values


# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()

female_Y = encoder.fit_transform(np.array(female_Y).reshape(-1,1)).toarray()
male_Y = encoder.fit_transform(np.array(male_Y).reshape(-1,1)).toarray()

nogender_X = np.concatenate((female_X, male_X))
nogender_Y = np.concatenate((female_Y, male_Y))

x_train, x_test, y_train, y_test = train_test_split(nogender_X, nogender_Y, random_state=0, test_size=0.20, shuffle=True)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

print(np.shape(x_train[0]))
print(x_train[0])

print(np.shape(np.reshape(x_train[0], (-1, 2))))

import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization, AveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

(68107, 58) (68107, 8) (17027, 58) (17027, 8)
(58,)
[-1.09247469e+02  2.85727250e+01  7.73544677e+00  1.00557306e+01
  9.98538413e-02 -8.79615193e+00 -6.81804130e+00 -5.85545605e+00
 -6.74060754e+00 -3.50669445e+00 -1.04472078e+00 -3.60818097e+00
 -2.95273611e+00 -1.98681789e+00 -1.88679836e+00 -2.31764245e+00
 -2.25652759e+00 -1.34698777e+00 -1.63102413e+00 -2.34688486e+00
 -2.12729669e+00 -5.75247875e-01 -2.51803556e+00 -2.80846675e+00
 -1.04429163e+00 -2.69909158e+00 -2.48723781e+00 -5.83938539e-01
 -7.33445792e-01 -7.90121392e-01  5.86400758e-01  1.24769854e+00
  3.24471398e-01 -9.55137169e-01 -1.69048133e+00  1.70124717e-01
  1.06024038e+00  2.25798623e-01 -1.27405707e+00 -2.31656908e+00
 -1.51546243e+00  9.16445108e-02  9.74495874e-01  2.47176776e+00
  4.30285147e+00  4.90206401e+00  3.08720122e+00  1.90315068e+00
  7.15769954e-01 -2.31201382e-01 -1.46847537e+00 -7.55827230e-01
 -1.81678258e-01  7.85999126e-01  1.06233736e+00  4.86968078e-01
  6.78759163e-01  4.56249921e-01]
(29,

In [3]:
def build_model(in_shape):
    
    model=Sequential()
    model.add(Conv1D(256, kernel_size=6, strides=1, padding='same', activation='relu', input_shape=(in_shape, 1)))
    model.add(AveragePooling1D(pool_size=4, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=6, strides=1, padding='same', activation='relu'))
    model.add(AveragePooling1D(pool_size=4, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=6, strides=1, padding='same', activation='relu'))
    model.add(AveragePooling1D(pool_size=4, strides = 2, padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv1D(64, kernel_size=6, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=4, strides = 2, padding = 'same'))
    
    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(units=8, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
        
    
    return model

def model_build_summary(mod_dim, tr_features, val_features, val_labels):
    model = build_model(mod_dim)
    model.summary()
    
    score = model.evaluate(val_features, val_labels, verbose = 1)
    accuracy = 100*score[1]
    
    return model

In [4]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=4, min_lr=0.000001)

batch_size = 32
n_epochs = 75

In [4]:
def show_graphs(history):
    epochs = [i for i in range(n_epochs)]
    fig , ax = plt.subplots(1,2)
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    test_acc = history.history['val_accuracy']
    test_loss = history.history['val_loss']

    fig.set_size_inches(30,12)
    ax[0].plot(epochs , train_loss , label = 'Training Loss')
    ax[0].plot(epochs , test_loss , label = 'Testing Loss')
    ax[0].set_title('Training & Testing Loss')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")

    ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
    ax[1].plot(epochs , test_acc , label = 'Testing Accuracy')
    ax[1].set_title('Training & Testing Accuracy')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    plt.show()

In [5]:
total_model = model_build_summary(x_train.shape[1], x_train, x_test, y_test)

2023-08-03 10:06:25.009666: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 58, 256)           1792      
                                                                 
 average_pooling1d (AverageP  (None, 29, 256)          0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 29, 128)           196736    
                                                                 
 average_pooling1d_1 (Averag  (None, 15, 128)          0         
 ePooling1D)                                                     
                                                                 
 conv1d_2 (Conv1D)           (None, 15, 128)           98432     
                                                                 
 average_pooling1d_2 (Averag  (None, 8, 128)           0

In [9]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=4, min_lr=0.000001)
batch_size = 32
n_epochs = 75
history = total_model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, validation_data=(x_test, y_test), callbacks=[rlrp])

Epoch 1/75
2129/2129 [==============================] - 32s 15ms/step - loss: 1.4794 - accuracy: 0.3991 - val_loss: 1.2097 - val_accuracy: 0.5092 - lr: 0.0010
Epoch 2/75
2129/2129 [==============================] - 31s 15ms/step - loss: 1.2482 - accuracy: 0.4949 - val_loss: 1.1472 - val_accuracy: 0.5487 - lr: 0.0010
Epoch 3/75
2129/2129 [==============================] - 30s 14ms/step - loss: 1.1793 - accuracy: 0.5265 - val_loss: 1.0943 - val_accuracy: 0.5629 - lr: 0.0010
Epoch 4/75
2129/2129 [==============================] - 29s 14ms/step - loss: 1.1347 - accuracy: 0.5442 - val_loss: 1.0402 - val_accuracy: 0.5805 - lr: 0.0010
Epoch 5/75
2129/2129 [==============================] - 30s 14ms/step - loss: 1.0876 - accuracy: 0.5624 - val_loss: 1.0205 - val_accuracy: 0.5899 - lr: 0.0010
Epoch 6/75
 956/2129 [============>.................] - ETA: 15s - loss: 1.0550 - accuracy: 0.5800

KeyboardInterrupt: 